In [1]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('train.csv')

In [3]:
x=df.drop('SalePrice',axis=1);
y=df['SalePrice']

In [4]:
num_cols=x.select_dtypes(include=['int64','float64']).columns
cat_cols = x.select_dtypes(include=['object']).columns

In [5]:
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


In [6]:
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [7]:
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_cols),
    ('cat', cat_transformer, cat_cols)
])

In [8]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [9]:
lasso_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lasso', Lasso(alpha=0.1))
])

lasso_model.fit(X_train, y_train)

C:\Users\singh\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 94309989047.78882, tolerance: 696659484.3571944
  model = cd_fast.sparse_enet_coordinate_descent(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrS...
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object'))])),
                ('lasso', Lasso(alpha=0.1))])

In [10]:
# Make predictions on the validation set
y_pred = lasso_model.predict(X_val)

In [11]:
from sklearn.metrics import mean_squared_error
import numpy as np
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print('RMSE:', rmse)

RMSE: 28327.740792673198


In [12]:
test_df = pd.read_csv('test.csv')

In [13]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [14]:
test_preds = lasso_model.predict(test_df)

In [15]:
# Create a submission file
submission = pd.DataFrame({'Id': test_df['Id'], 'SalePrice': test_preds})
submission.to_csv('submission.csv', index=False)

In [16]:
from sklearn.ensemble import RandomForestRegressor

In [22]:
regressor = RandomForestRegressor(n_estimators=100, random_state=43)

In [23]:
random_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('Random', RandomForestRegressor(n_estimators=100, random_state=0))
])

random_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrS...
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object'))])),
                ('Random', RandomForestRegressor(random_state=0))])

In [24]:
y_pred = random_model.predict(X_val)

In [25]:
from sklearn.metrics import mean_squared_error
import numpy as np
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print('RMSE:', rmse)

RMSE: 29073.232947439672


In [21]:
# Create a submission file
submission = pd.DataFrame({'Id': test_df['Id'], 'SalePrice': test_preds})
submission.to_csv('submissionRandom.csv', index=False)